In [4]:
from pathlib import Path

data_dir = Path("../data")
file = data_dir / "The-Gale-Encyclopedia-of-Medicine-3rd-Edition.pdf"

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

def extract_text_from_pdf(path):
    loader = DirectoryLoader(
        path=path,
        glob="*.pdf",
        loader_cls=PyPDFLoader 
    )

    document = loader.load()

    return document

In [6]:
# extracted_data = extract_text_from_pdf(data_dir)

In [7]:
# extracted_data[1]

In [8]:
# only needs source & page_content

from langchain.schema import Document


def filter_source_and_page_content(docs):
    filtered_docs: list[Document] = []

    for doc in docs:
        src = doc.metadata.get("source")
        filtered_docs.append(
            Document(
                metadata={"source": src},
                page_content=doc.page_content
            )
        )
    
    return filtered_docs

In [9]:
# filtered_docs = filter_source_and_page_content(extracted_data)
# filtered_docs[1]

In [10]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# def chunk_content(doc):
#     text_splitter = RecursiveCharacterTextSplitter(
#         chunk_size=512,
#         chunk_overlap=64,
#         length_function=len
#     )

#     text_chunks = text_splitter.split_documents(doc)
#     return text_chunks

In [11]:
# text_chunks = chunk_content(filtered_docs)
# len(text_chunks)

In [12]:
import torch

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [13]:
from langchain_huggingface import HuggingFaceEmbeddings

def download_embedding():
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs={"device": DEVICE}
    )

    return embeddings

In [14]:
embedding = download_embedding()

/home/cr1xx/Documents/roadmap/medbot/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
vector = embedding.embed_query("hello world")
len(vector)

384

In [16]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [ ]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

if "GROQ_API_KEY" not in os.environ:
    print("Please set the GROQ_API_KEY environment variable.")
    exit()

In [18]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)

In [19]:
from pinecone import ServerlessSpec, CloudProvider, AwsRegion

index_name = "medbot"

if not pc.has_index(index_name):
    pc.create_index(name=index_name,
                    dimension=384,
                    metric="cosine",
                    spec=ServerlessSpec(
                        cloud=CloudProvider.AWS, 
                        region=AwsRegion.US_EAST_1 # pinecone free tier only have this region available
                    ) 
    )

index = pc.Index(index_name)

In [20]:
# add embeddings to index

# from langchain_pinecone import PineconeVectorStore

# doc_search = PineconeVectorStore.from_documents(
#     documents=text_chunks,
#     embedding=embedding,
#     index_name=index_name
# )

In [21]:
# load existing index

from langchain_pinecone import PineconeVectorStore

doc_search = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

In [22]:
# add more data to existing index

lorem = Document(
    page_content="Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.",
    metadata={"source": "Lipsum"}
)

# doc_search.add_documents(documents=[lorem,])

In [23]:
retriever = doc_search.as_retriever(search_type="similarity", search_kwargs={"k": 3}) # top 3

In [24]:
retrieved_docs = retriever.invoke("What is acne?")
retrieved_docs

[Document(id='f3ad989f-2264-4c87-a3ef-0322b15a8f6e', metadata={'source': '../data/The-Gale-Encyclopedia-of-Medicine-3rd-Edition.pdf'}, page_content='pimples on the face, chest, and back. It occurs when\nthe pores of the skin become clogged with oil, dead\nskin cells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne,\nis the most common skin disease. It affects nearly 17\nmillion people in the United States. While acne can\narise at any age, it usually begins atpuberty and wor-\nsens during adolescence. Nearly 85% of people\ndevelop acne at some time between the ages of 12-25\nyears. Up to 20% of women develop mild acne. It is'),
 Document(id='52392759-da58-4e47-b9cf-24b6fbb97f57', metadata={'source': '../data/The-Gale-Encyclopedia-of-Medicine-3rd-Edition.pdf'}, page_content='turnover (death and replacement) of skin cells.\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous glands\nbecome inflamed.

In [25]:
from langchain_groq import ChatGroq

chat_model = ChatGroq(model_name="groq/compound-mini", temperature=0)

In [26]:
# from langchain_openai import ChatOpenAI

# chat_model = ChatOpenAI(model="gpt-4o-mini") # gpt-4o

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are a medical assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Make you answer concise by having maximum of "
    "three sentences." # limit 
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [28]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

question_answer_chain = create_stuff_documents_chain(chat_model, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [29]:
response = rag_chain.invoke({"input": "Tips on preventing acne."})
print(response["answer"])

To minimize acne flare-ups, gently wash affected areas once or twice a day with a non-abrasive cleanser. A diet rich in nutrients such as essential fatty acids, vitamin B complex, zinc, vitamin A, and chromium may also help. Additionally, consider Chinese herbal remedies like cnidium seed and honeysuckle flower, but consult a physician or nutritionist for proper amounts.
